In [73]:
import requests
import pandas as pd
import json
from dateutil import parser

In [74]:
API_KEY = "bb2c9611a7addc2bb78c7c3433c40171-097edf996c282e83518e13ad2f045839"
ACCOUNT_ID = "101-002-27231304-001"
OANDA_URL = 'https://api-fxpractice.oanda.com/v3'


In [75]:
# Request has a session object, help maintain persisteng connection
session = requests.Session()

In [76]:
session.headers.update({
    "Authorization" :f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [77]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [78]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [79]:
response = session.get(url, params= params, data = None, headers=None)

In [80]:
response.status_code

200

In [81]:
data= response.json()
instruments_list = data['instruments']

In [82]:
len(instruments_list)

123

In [83]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [84]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [85]:
instrument_dict= {}
for i in instruments_list:
    key = i['name']
    instrument_dict[key] = {k : i[k] for k in key_i}

In [86]:
instrument_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [87]:
with open('../data/instruments.json', 'w') as f:
    f.write(json.dumps(instrument_dict, indent=2))

In [88]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data = None, headers= None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
            

    return response.status_code, data



In [89]:
# time
# volume
# only want complete candles


def get_candles_df(data):
    if len(data) ==0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    final_data = []
    for candle in data:
        if candle['complete'] ==False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"]= float(candle[price][oh])   
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df


In [90]:
def create_data_file(pair_name, count = 10, granularity ="H1"):
    code, data = fetch_candles(pair_name, count, granularity )
    if code != 200:
        print("Failed", pair_name)
        return
    if len(data) ==0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()} ")
    

In [91]:
code, data = fetch_candles("EUR_USD", count = 10, granularity="H4" )
candles_df = get_candles_df(data)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (1562011658.py, line 1)

In [92]:
create_data_file("EUR_USD", count = 10, granularity="H4")

EUR_USD H4 9 candles, 2023-11-15 10:00:00+00:00 2023-11-16 18:00:00+00:00 


In [94]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD', 'CAD', 'AUD']

In [98]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in instrument_dict:
            for g in ["H1", "H4"]:
                create_data_file(pair, count = 4001, granularity=g)

EUR_USD H1 4000 candles, 2023-03-29 07:00:00+00:00 2023-11-16 23:00:00+00:00 
EUR_USD H4 4000 candles, 2021-04-27 05:00:00+00:00 2023-11-16 18:00:00+00:00 
EUR_GBP H1 4000 candles, 2023-03-29 07:00:00+00:00 2023-11-16 23:00:00+00:00 
EUR_GBP H4 4000 candles, 2021-04-27 05:00:00+00:00 2023-11-16 18:00:00+00:00 
EUR_JPY H1 4000 candles, 2023-03-29 07:00:00+00:00 2023-11-16 23:00:00+00:00 
EUR_JPY H4 4000 candles, 2021-04-26 21:00:00+00:00 2023-11-16 18:00:00+00:00 
EUR_CHF H1 4000 candles, 2023-03-29 07:00:00+00:00 2023-11-16 23:00:00+00:00 
EUR_CHF H4 4000 candles, 2021-04-27 05:00:00+00:00 2023-11-16 18:00:00+00:00 
EUR_NZD H1 4000 candles, 2023-03-29 06:00:00+00:00 2023-11-16 23:00:00+00:00 
EUR_NZD H4 4000 candles, 2021-04-27 05:00:00+00:00 2023-11-16 18:00:00+00:00 
EUR_CAD H1 4000 candles, 2023-03-29 07:00:00+00:00 2023-11-16 23:00:00+00:00 
EUR_CAD H4 4000 candles, 2021-04-27 05:00:00+00:00 2023-11-16 18:00:00+00:00 
EUR_AUD H1 4000 candles, 2023-03-29 06:00:00+00:00 2023-11-16 23